# Analysis of Classification Results – TSST vs. f-TSST (per Phase)

## Imports and Helper Functions

In [ ]:
import json
import re
from pathlib import Path

import biopsykit as bp
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
from biopsykit.classification.analysis import (
    predictions_as_df,
    predict_proba_from_estimator,
    plot_conf_matrix,
    plot_conf_matrix_proba,
)
from biopsykit.classification.model_selection import SklearnPipelinePermuter
from fau_colors import cmaps, register_fausans_font

from stresspose_analysis.datasets import MainStudyDataset
from stresspose_analysis.classification.utils import (
    feature_data_long_to_wide,
    shap_values_per_fold,
    get_shap_feature_importances,
)
from stresspose_analysis.plotting.mainstudy import shap_feature_importances_plot


%load_ext autoreload
%autoreload 2
%matplotlib widget

In [ ]:
register_fausans_font()
plt.close("all")

palette = sns.color_palette(cmaps.faculties)
sns.set_theme(context="notebook", style="ticks", palette=palette)

plt.rcParams["figure.figsize"] = (8, 4)
plt.rcParams["pdf.fonttype"] = 42
plt.rcParams["mathtext.default"] = "regular"
plt.rcParams["font.family"] = "sans-serif"
plt.rcParams["font.sans-serif"] = "FAUSans Office"


palette

## Setup Paths

In [ ]:
deploy_type = "local"

In [ ]:
config_path = Path("../../../config.json")
config_dict = json.load(config_path.open(encoding="utf-8"))

base_path = Path(config_dict[deploy_type]["base_path"])
dataset = MainStudyDataset(base_path)
dataset

In [ ]:
label_mapping = {"tsst": "TSST", "ftsst": "f-TSST"}

## Load Data

In [ ]:
classification_type = "per_phase"

In [ ]:
root_path = Path("../../..")
input_path = root_path.joinpath(f"output/classification")
output_path = root_path.joinpath("results")

In [ ]:
img_path = output_path.joinpath("plots")
bp.utils.file_handling.mkdirs([img_path])

In [ ]:
pickle_files = sorted(input_path.glob("*.pkl"))

In [ ]:
feature_file = sorted(input_path.glob(f"movement_features_per_phase_for_classification.csv"))[0]

## Load `PipelinePermuter`

In [ ]:
pipeline_permuter = SklearnPipelinePermuter.merge_permuter_instances(pickle_files)
# pipeline_permuter = SklearnPipelinePermuter.from_pickle(pickle_files[0])

### Features

In [ ]:
data = bp.io.load_long_format_csv(feature_file)
data = data.rename(index=label_mapping, level="condition")
data.head()

In [ ]:
data_wide = feature_data_long_to_wide(data, index_levels_out=["subject", "condition"])
data_wide.head()

## Display Results

### Metric Summary

The summary of all relevant metrics (performance scores, confusion matrix, true and predicted labels) of the **best pipelines** for each fold (i.e., the `best_estimator_` parameter of each inner `cv` object), evaluated for each evaluated pipeline combination.

In [ ]:
metric_summary = pipeline_permuter.metric_summary(additional_metrics=["f1_score", "precision"], pos_label="tsst")
metric_summary = metric_summary.sort_values(by="mean_test_accuracy", ascending=False)
metric_summary.head()

### Best Pipeline per Classifier

In [ ]:
best_clfs = metric_summary.groupby("pipeline_clf", group_keys=False).apply(
    lambda df: df.sort_values(by="mean_test_accuracy", ascending=False).iloc[[0]]
)
best_clfs = best_clfs.sort_values(by="mean_test_accuracy", ascending=False)
best_clfs = best_clfs.droplevel(level="pipeline_remove_var")
best_clfs

In [ ]:
latex_output = pipeline_permuter.metric_summary_to_latex(
    data=best_clfs,
    pipeline_steps=["pipeline_scaler", "pipeline_reduce_dim", "pipeline_clf"],
    clines=None,
    sparse_index=False,
    highlight_best=True,
    si_table_format="table-format = 2.1(3)",
    caption=r"\textit{Main Study}: Mean $\pm$ standard deviation of classification performance metrics over the 5-fold model evaluation CV with features separately computed over the \textit{Interview} and \textit{Mental Arithmetics} phases of the (f-)TSST, respectively. For each evaluated classifier, the classification pipeline combination with the highest mean accuracy is shown. The classification pipelines scoring the highest metrics are highlighted in \textbf{bold}.",
    label="tab:classification_results_per_phase_mainstudy",
)
# some dirty manual postprocessing of output
latex_output = re.sub(r"\\cline{1-4} \\cline{2-4}\n\\bottomrule", r"\\bottomrule", latex_output)
latex_output = re.sub(r"\\cline{1-4} \\cline{2-4}", r"\\cline{1-4}", latex_output)
latex_output = re.sub(
    r"{} & {} & {}", r"{Scaler} & {\\makecell[lc]{Feature\\\\ Selection}} & {Classifier}", latex_output, count=1
)
latex_output = re.sub(
    r"{Scaler} & {\\makecell\[lc\]{Feature\\\\ Selection}} & {Classifier} & {} & {} & {} \\\\\n", r"", latex_output
)

print(latex_output)

## Detailed Analysis

In [ ]:
selected_pipeline = ("VarianceThreshold", "MinMaxScaler", "SelectFromModel", "RandomForestClassifier")

labels = ["TSST", "f-TSST"]

In [ ]:
best_estimator_summary = pipeline_permuter.best_estimator_summary()
best_estimator_summary.head()

In [ ]:
predictions = predictions_as_df(pipeline_permuter, data_wide, selected_pipeline, label_mapping)
predictions = predictions.join(dataset.condition_order).join(dataset.cort_non_responder)
predictions = predictions.join(
    bp.utils.dataframe_handling.apply_codebook(pd.DataFrame(dataset.gender), dataset.codebook)
)
predictions.head()

### Confusion Matrix

In [ ]:
fig, ax = plt.subplots(figsize=(3, 3))
plot_conf_matrix(predictions, labels, label_name="condition", ax=ax)
fig.tight_layout(pad=0)

fig.savefig(img_path.joinpath("img_confusion_matrix_per_phase_mainstudy.pdf"), transparent=True)

#### Confusion Matrix for Paper Teaser Figure

In [ ]:
with sns.plotting_context("talk"):
    fig, ax = plt.subplots(figsize=(4, 4))
    plot_conf_matrix(
        predictions, labels={"TSST": "Stress", "f-TSST": "No Stress"}, label_name="condition", despine=False, ax=ax
    )
    fig.tight_layout(pad=0.1)

    fig.savefig(img_path.joinpath("img_confusion_matrix_teaserfigure.pdf"), transparent=True)

### Confusion Matrix by Condition Order

In [ ]:
fig, axs = plt.subplots(figsize=(6, 3), ncols=2)

for (key, df), ax in zip(predictions.groupby("condition_order"), axs):
    plot_conf_matrix(df, labels, label_name="condition", ax=ax)
    ax.set_title(key)

fig.tight_layout(pad=0, w_pad=1)

### Confusion Matrix by Cortisol Non-Responder

In [ ]:
fig, axs = plt.subplots(figsize=(6, 3), ncols=2)

for (key, df), ax in zip(predictions.groupby("non_responder"), axs):
    plot_conf_matrix(df, labels, ax=ax)
    ax.set_title(f"Non-Responder: {key}")

fig.tight_layout(pad=0, w_pad=1)

### Confusion Matrix by Gender

In [ ]:
fig, axs = plt.subplots(figsize=(6, 3), ncols=2)

for (key, df), ax in zip(predictions.groupby("Gender"), axs):
    plot_conf_matrix(df, labels, ax=ax)
    ax.set_title(f"Gender: {key}")

fig.tight_layout(pad=0, w_pad=1)

### Prediction Probability

In [ ]:
predictions_proba = predict_proba_from_estimator(
    pipeline_permuter, data_wide, selected_pipeline, label_col="condition", column_names=label_mapping
)

In [ ]:
fig, ax = plt.subplots(figsize=(3, 3))

plot_conf_matrix_proba(predictions_proba, labels=labels, label_col="condition", ax=ax)

fig.tight_layout(pad=0)

In [ ]:
predictions_proba_cond = predictions_proba.join(dataset.condition_order).join(dataset.cort_non_responder)
predictions_proba_cond = predictions_proba_cond.set_index(["condition_order", "non_responder"], append=True)

### Prediction Probability by Condition Order

In [ ]:
fig, axs = plt.subplots(figsize=(6, 3), ncols=2)

for (key, df), ax in zip(predictions_proba_cond.groupby("condition_order"), axs):
    plot_conf_matrix_proba(df, labels=labels, label_col="condition", ax=ax)
    ax.set_title(key)

fig.tight_layout(w_pad=1)

### Prediction Probability by Cortisol Non-Responder

In [ ]:
fig, axs = plt.subplots(figsize=(6, 3), ncols=2)

for (key, df), ax in zip(predictions_proba_cond.groupby("non_responder"), axs):
    plot_conf_matrix_proba(df, labels=labels, label_col="condition", ax=ax)
    ax.set_title(f"Non-Responder: {key}")

fig.tight_layout(w_pad=1)

## Get Feature Importances using SHAP

In [ ]:
shap_values, data_out = shap_values_per_fold(pipeline_permuter, selected_pipeline, data_wide)

### SHAP Summary Plot

In [ ]:
fig, ax = plt.subplots()
shap_feature_importances_plot(shap_values[0], features=data_out, plot_size=(12, 5))

fig.savefig(img_path.joinpath("img_feature_shap_importance_per_phase_mainstudy.pdf"), transparent=True)

### Most Important Features

In [ ]:
feature_importances = get_shap_feature_importances(shap_values[0], data)

In [ ]:
n = 40
top_features = feature_importances.head(n=n)
top_features.head()

#### Per Feature Type

In [ ]:
number_features = pd.DataFrame(top_features.groupby("feature_type").size(), columns=["Count"]).T
number_features["total"] = [len(top_features)]
number_features

#### Per Channel

In [ ]:
number_features = pd.DataFrame(top_features.groupby("channel").size(), columns=["Count"]).T
number_features["total"] = [len(top_features)]
number_features

#### Per Metric

In [ ]:
number_features = pd.DataFrame(top_features.groupby("metric").size(), columns=["Count"]).T

fft_features = ["fft_aggregated_centroid", "fft_aggregated_kurtosis", "fft_aggregated_variance", "fft_aggregated_skew"]
number_features = number_features.assign(
    fft=number_features.reindex(fft_features, axis=1).sum().sum().astype(int), total=len(top_features)
)
number_features = number_features.drop(columns=fft_features, errors="ignore")

number_features

#### Per Body Part

In [ ]:
upper_extremities = [
    "LeftHand_RightHand",
    "LeftHand_Head",
    "RightHand_Head",
    "LeftHand",
    "UpperExtremities",
]
lower_extremities = ["LowerExtremities", "LeftFoot_RightFoot"]
trunk = [
    "T8",
    "Trunk",
]
head = [
    "Head",
]
total_body = ["TotalBody", "CenterMass"]

feature_counts_body_part = top_features.groupby("body_part").size().sort_values(ascending=False)
feature_counts_body_part = pd.DataFrame(feature_counts_body_part, columns=["Counts"])

print_dict = {
    "Head": head,
    "Upper Extremities": upper_extremities,
    "Lower Extremities": lower_extremities,
    "Trunk": trunk,
    "Total Body": total_body,
}

for key, value in print_dict.items():
    print(f"{key}: {int(feature_counts_body_part.reindex(value).sum().sum())}")

feature_counts_body_part

In [ ]:
features_plot = feature_importances.head(n=5).index
data_unstack = data["data"].unstack(["subject", "condition"]).T
data_plot = data_unstack.loc[:, features_plot]
data_plot.columns = ["-".join(col) for col in data_plot.columns]

pairgrid, features = bp.plotting.feature_pairplot(data=data_plot, hue="condition")
display(features)